In [4]:
df <- read.csv("test_data/gene_batch_search.csv")
df

X,Variant.ID,Gene,Consequence,Annotation,Flags,Allele.Count,Allele.Number,Allele.Frequency,Number.of.Homozygotes,⋯,Amish,Latino,Ashkenazi.Jewish,East.Asian,European..Finnish.,European..non.Finnish.,Other,South.Asian,Middle.Eastern,Number.of.Hemizygotes
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,4-987010-C-G,IDUA,c.-75C>G,5_prime_UTR_variant,[],1,149172,6.703671e-06,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,1.486945e-05,0.0000000000,0,0.000000000,NA
1,4-987013-G-A,IDUA,c.-72G>A,5_prime_UTR_variant,[],3,151256,1.983392e-05,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,9.606148e-05,2.949070e-05,0.0000000000,0,0.000000000,NA
2,4-987014-C-T,IDUA,c.-71C>T,5_prime_UTR_variant,[],37,152092,2.432738e-04,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,1.884304e-04,4.411505e-04,0.0004780115,0,0.000000000,NA
3,4-987021-T-C,IDUA,c.-64T>C,5_prime_UTR_variant,[],2,146758,1.362788e-05,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,2.992847e-05,0.0000000000,0,0.000000000,NA
4,4-987023-C-G,IDUA,c.-62C>G,5_prime_UTR_variant,[],1,152126,6.573498e-06,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.0000000000,0,0.000000000,NA
5,4-987023-C-T,IDUA,c.-62C>T,5_prime_UTR_variant,[],1,152126,6.573498e-06,0,⋯,0,6.543646e-05,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.0000000000,0,0.000000000,NA
6,4-987026-C-T,IDUA,c.-59C>T,5_prime_UTR_variant,[],2,152122,1.314734e-05,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.0000000000,0,0.000000000,NA
7,4-987027-C-G,IDUA,c.-58C>G,5_prime_UTR_variant,[],1,152154,6.572289e-06,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,1.470415e-05,0.0000000000,0,0.000000000,NA
8,4-987027-C-A,IDUA,c.-58C>A,5_prime_UTR_variant,[],1,152154,6.572289e-06,0,⋯,0,0.000000e+00,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.0000000000,0,0.000000000,NA


In [37]:
get_reoordered_df <- function(df){
    col_idx_homo <- grep("Number.of.Homozygotes", names(df))
    col_idx_hemi <- grep("Number.of.Hemizygotes", names(df))
    col_len <- length(names(df))
    
    return(df[, c(1:col_idx_homo,col_idx_hemi,(col_idx_homo+1):col_len)[-col_idx_hemi-1]])
}



get_pop_dfs <- function(df=NULL, freq_threshold=0.001){
    
    col_idx_alternative <- grep("Alternative", names(df))
    population_indexes <- c((col_idx_alternative+1):length(names(df)))
    populations_dfs = c()
    
    i <- 1
    for (pop_idx in population_indexes){
        df_temp <- df[, c(1:col_idx_alternative,pop_idx)]
        populations_dfs[[i]] <- df_temp[df_temp[,ncol(df_temp)] > freq_threshold, ]
        i <- i + 1
    }
    
    return(populations_dfs)
}

In [38]:
df <- get_reoordered_df(df)
pop_dfs <- get_pop_dfs(df)

In [41]:
pop_dfs[[3]]

,X,Variant.ID,Gene,Consequence,Annotation,Flags,Allele.Count,Allele.Number,Allele.Frequency,Number.of.Homozygotes,Number.of.Hemizygotes,Source,Chromosome,Location,Reference,Alternative,Latino
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
27,26,4-987095-T-C,IDUA,p.Leu4Pro,missense_variant,[],96,152004,0.0006315623,0,NA,Genome,4,987095,T,C,0.001048218
32,31,4-987108-C-A,IDUA,p.Pro8Gln,missense_variant,[],60531,151804,0.3987444336,12356,NA,Genome,4,987108,C,A,0.484432500
35,34,4-987114-G-GCTGGCGCTC,IDUA,p.Leu13_Ala15dup,inframe_insertion,[],58,152034,0.0003814936,1,NA,Genome,4,987114,G,GCTGGCGCTC,0.003471313
47,46,4-987144-G-A,IDUA,p.Arg20His,missense_variant,[],60646,151784,0.3995546303,12409,NA,Genome,4,987144,G,A,0.482817400
62,61,4-987183-T-G,IDUA,p.His33Gln,missense_variant,[],129624,152032,0.8526099768,55703,NA,Genome,4,987183,T,G,0.819730100
136,135,4-987885-G-A,IDUA,p.Ala79Thr,missense_variant,[],1291,152198,0.0084823716,15,NA,Genome,4,987885,G,A,0.002289677
141,140,4-987896-C-G,IDUA,p.His82Gln,missense_variant,[],571,152240,0.0037506569,1,NA,Genome,4,987896,C,G,0.004774987
158,157,4-987946-C-T,IDUA,p.Thr99Ile,missense_variant,[],834,152238,0.0054782643,7,NA,Genome,4,987946,C,T,0.001634842
162,161,4-987955-C-T,IDUA,c.299+6C>T,splice_region_variant,[],1405,152140,0.0092349152,14,NA,Genome,4,987955,C,T,0.007325049


In [15]:
pop_dfs[1][pop_dfs[1] == 'water',]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


X,Variant.ID,Gene,Consequence,Annotation,Flags,Allele.Count,Allele.Number,Allele.Frequency,Number.of.Homozygotes,⋯,African,Amish,Latino,Ashkenazi.Jewish,East.Asian,European..Finnish.,European..non.Finnish.,Other,South.Asian,Middle.Eastern
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
